# 1. Introduction

This module will cover intermediate and advanced Python topics -- installing and creating packages, data loading and saving, visualization tools, multiprocessing, functions and classes, and exceptions and debugging.


## 1.1 Anaconda & Spyder

The great thing about Python is that it has so many packages (also called libraries or modules) written for it. But they're a pain to install yourself. The easiest way to install a "scientific Python" installation on your computer is with Anaconda:

https://www.anaconda.com/distribution/#download-section

Everything you need is preinstalled on the OCNC virtual machines, so if you're using that, you don't need to do anything. However, you are encouraged to set it up on your local computer, since you probably won't keep using the virtual machine after OCNC!

Anaconda includes an "interactive development environment" (IDE) called Spyder, which provides a Matlab-like interface to Python. For big projects, it's much more flexible than a Jupyter notebook.

<div class="alert alert-block alert-info">
<b>Exercise 1.1:</b><br>
    Start Spyder.
</div>


## 1.2 Pylab

One useful way of using Python is by including the following at the top of your file:

`import pylab as pl`

or

`from pylab import *`

Pylab is a package that contains all the most commonly used scientific functions. The second line is basically just shorthand for

```python
from numpy import * # numerical functions like arrays, log, exp
from scipy import * # scientific functions like interpolation
from matplotlib.pyplot import * # plotting functions
```

Note that `import pylab as pl` is more "pythonic" (and is strongly encouraged!), but `from pylab import *` lets you have the most Matlab-like environment. For example, the following is valid Python _or_ Matlab code!

```python
from pylab import * # comment out in matlab
randomnumbers = randn(50,10);
cumulativenumbers = cumsum(abs(randomnumbers));
plot(cumulativenumbers);
```

(Caveat: these give slightly different results though!)

<div class="alert alert-block alert-info">
<b>Exercise 1.2:</b><br>
    (A) Run the code block in both Matlab and Python. <br>
    (B) Read the Python documentation on the "cumsum" function ("help(cumsum)") and figure out how to make Python match Matlab.
</div>


## 1.3 Installing packages

Numpy, Pylab, and so forth are examples of _packages_. The easiest way to install packages in Python is with `pip` (package installer for python). This searches http://pypi.org for a package with that name, downloads it, and installs it, while you sit back and sip tea. Usually you would just do `pip install the_package_i_want`, but on the OCNC virtual machines you need a slightly different syntax: `python3 -m pip install the_package_i_want`.

We will use Sciris a bit later, but as a small preview, Sciris includes a data structure called an `odict`, which is another kind of ordered dictionary that also behaves like a list wherever possible:

```python
import pylab as pl
import sciris as sc

citations_dict = {'kenji doya':6363, 'erik de schutter':6366, 'eve marder': 27480}
citations_odict = sc.odict({'kenji doya':6363, 'erik de schutter':6366, 'eve marder': 27480})

total_citations = pl.array(citations_dict.values()).sum()
total_citations = citations_odict[:].sum()

alphabetized_dict = {key:citations_dict[key] for key in sorted(list(citations_dict.keys()))}
alphabetized_odict = citations_odict.sorted()

first_entry = alphabetized_dict[list(alphabetized_dict.keys())[0]]
first_entry = alphabetized_odict[0]
```

<div class="alert alert-block alert-info">
<b>Exercise 1.3:</b><br>
    (A) If you're using the virtual machine: type "python3 -m pip install sciris" to install the Sciris package. Otherwise: type "pip install sciris".<br>
    (B) Run the example code above. Extra credit (warning, difficult!): you can sort an odict by value with "citations_odict.sorted(sortby='values')". How would you create a regular dictionary sorted by value?
</div>


## 1.4 Creating packages

If you're working on a big project, you don't want everything in one file. To use functions from one file in another, you can import that file, just as you would `numpy` or `pylab`. For example, if you call your file `myfunction.py`, and inside that file you have defined `def myfunction()`, you would use it as follows:

```python
import myfunction as myf
output = myf.myfunction()
```

Note: this syntax of `nameofthing.nameofthing()` is frustratingly common in Python, but you will learn to see beauty in it. (Maybe.)

<div class="alert alert-block alert-info">
<b>Exercise 1.4:</b><br>
    (A) Convert the block of code from Exercise 1.2 into a function that returns `cumulativenumbers` (but doesn't plot), and save this as a separate file (e.g. "exercise1.py".<br>
    (B) Create a new file (e.g. "exercise1plot.py") that imports this function, calls it, and then plots the results.
</div>


## 1.5 Classes and methods

There are two basic approaches to programming: functional programming and object-oriented programming (OOP). One isn't "better" than the other, it depends on what you're trying to do. In general, you have data, and functions that do things on the data. In functional programming, you keep your data together in one place, and you keep your functions together in another place. In OOP, you keep data and functions together in an _object_.

Let's make it more concrete. Let's say you have you have a worm, a pigeon, and a cat. If you care about their _shared_ properties and functions that apply to all of them, you want to take a functional programming approach, for example:

```python
# Functional approach

masses = {'worm':0.002, 'pigeon':0.086, 'cat':7.3} # in kg
velocities = {'worm':0.01, 'pigeon':9.6, 'cat':4.4} # in m/s

def energy(mass, velocity):
    return 0.5*mass*velocity**2

print('Maximum kinetic energy for:')
for animal in masses.keys():
    KE = energy(masses[animal], velocities[animal])
    print(f'{animal}: {KE}')
```

However, it might also make more sense to treat each animal as an _object_ with properties/attributes (data) and methods (functions). That would look like this:

```python
# Object-oriented
    
class Animal():
    
    def __init__(self, name, mass, velocity):
        self.name = name
        self.mass = mass
        self.velocity = velocity
        
    def energy(self):
        return 0.5*self.mass*self.velocity**2
    
    
worm   = Animal(name='worm',   mass=0.002, velocity=0.01) # shorthand: Animal('worm', 0.002, 0.01)
pigeon = Animal(name='pigeon', mass=0.086, velocity=9.6)
cat    = Animal(name='cat',    mass=7.3,   velocity=4.4)

print('Maximum kinetic energy for:')
for animal in [worm, pigeon, cat]:
    print(f'{animal.name}: {animal.energy()}')
```

Note that these two approaches are **exactly equivalent**, it's just a matter of which one works better for you. OOP is almost always more work to set up initially, but can make life easier later. That said, for most of you most of the time, functional programming probably makes the most sense.


## 1.6 Multiprocessing

Multiprocessing means running on multiple cores of your computer simultaneously. If you have a big simulation, for example, then if you can run it across 8 cores, it should finish 8 times faster.

This code gives an example of a computationally intensive function run in an "embarrassingly parallel" way (each run is completely independent). It should take 10-20 seconds in serial (depending on your computer), and be 2-8 times faster in parallel.

```python
# Imports
import pylab as pl
import sciris as sc
import multiprocessing as mp

# Set parameters
xmin = 0
xmax = 10
npts = 200
std = 0.1
repeats = 100000
x = pl.linspace(xmin, xmax, npts)
run_series = True
run_manual = True
run_sciris = True
do_plot = True

# Define the random wave generator
def randgen(std):
    pl.seed(int(std*100))
    a = 0.2*pl.cos(x)*pl.sqrt(repeats)
    b = pl.zeros(npts)
    for r in range(repeats):
        b += (0.5-pl.rand(npts))*std
    return a+b


# Set arguments for the parallel run
noisevals = pl.linspace(0,1,21)

# Run in series
if run_series:
    sc.tic()
    data = []
    for noiseval in noisevals:
        output = randgen(noiseval)
        data.append(output)
    sc.toc()

# Run in parallel -- manual way
if run_manual:
    sc.tic() 
    multipool = mp.Pool(processes=mp.cpu_count())
    data = multipool.map(randgen, noisevals)
    multipool.close()
    multipool.join()
    sc.toc()

# Run in parallel -- easy way
if run_sciris:
    sc.tic() 
    data = sc.parallelize(randgen, noisevals)
    sc.toc()

# Create 3D plot
if do_plot:
    sc.surf3d(pl.array(data))
```

## 1.7 Debugging

If you write code, you will also write bugs. Thankfully, debugging is easy in Spyder.

Consider this truly awful code:

```python
def buggy_function1(vector, multiplier):
    output = {'max': max(vector*multiplier)}
    return output

def buggy_function2():
    calculation = buggy_function1(vector=[3,4,5], multiplier=3) # should return 15, right?
    output = calculation['maximum']
    return output

output = buggy_function2()
print(output)
```

There are two main kinds of debugging: figuring out why something crashed, and figuring out why something isn't as you expected. For the first, run your code, and after it crashes, type `%debug` in the console; that will put you in your program on the line it crashed. For the second, double-click on the line number you want to pause at; this will put in a _breakpoint_. Then, instead of running your code the normal way (F5), run it in debug mode (Ctrl+F5, or from the menu). It should then stop execution at the breakpoint.

<div class="alert alert-block alert-info">
<b>Exercise 1.7:</b><br>
    (A) Using the "%debug" command, find and fix the bug in buggy_function2().<br>
    (B) Using a breakpoint on line 3, find and fix the bug in buggy_function1().
</div>


## 1.8 Python 2 vs. Python 3

Don't use Python 2.



# 2. Data saving and loading

One of the great things about Python is that it can load/save data in just about any format. Here we will just look at a couple.

Except for large files (say >10 MB), it's often easiest just to read/write data files as text. Files in a plain text format can be read by anything and written by anything (more or less).

## 2.1 With numpy

First, make sure you have the file `data/Okinawa_weather_data.csv`. You would hope you can do something simple, like:

```python
import pylab as pl
filename = 'data/Okinawa_weather_data.csv'
data = pl.loadtxt(filename)
```

But that doesn't work (although it almost does).

<div class="alert alert-block alert-info">
<b>Exercise 2.1A:</b><br>
    By reading the documentation for loadtxt(), fix that code so the file will load (hint: you need to add two keyword arguments).
</div>

Now that we've loaded the data, we want to _do_ something with it. We can plot the 5 variables of interest with:

```python
pl.figure() # not strictly necessary, python will create a figure automatically
pl.plot(data[:,5:10])
```

We can also for example plot temperature vs. humidity:

```python
pl.figure() # this is necessary, since otherwise will plot over the other figure
pl.scatter(data[:,5],data[:,6])
pl.xlabel('temperature (deg C)')
pl.ylabel('rel. humidity')
```

<div class="alert alert-block alert-info">
<b>Exercise 2.1B:</b><br>
    Using the code above, plot the data. For the first figure, modify the plot command so the line width is 3.
</div>


## 2.2 With Pandas

If you are regularly working with tabular files like this, you should have a look at the [`pandas` library](http://pandas.pydata.org/), a powerful library built exactly for this use case:

```python
import pandas as pd
pd_data = pd.read_csv('data/Okinawa_weather_data.csv', sep=';')
print(pd_data)
```

In contrast with `numpy`, `pandas` reads data into a **dataframe**. This is an extremely powerful data structure with a steep learning curve. A full explanation is beyond the scope of this tutorial, but see the [documentation](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) and [a good tutorial](https://www.datacamp.com/community/tutorials/pandas-tutorial-dataframe-python) for more information.

To give a brief example of the possibilities of dataframes, to group the data by days and plot the mean and standard deviation of temperature over a day, run the following:

```python
grouped_data = pd_data.groupby('Day').aggregate(['mean', 'std'])
temp = grouped_data['Temperature  [2 m above gnd]']
pl.plot(temp.index, temp['mean'])
pl.fill_between(temp.index, temp['mean'] - 2*temp['std'], temp['mean'] + 2*temp['std'], alpha=0.2)
pl.xlabel('Day')
pl.ylabel('Temperature (C)')
```

<div class="alert alert-block alert-info">
<b>Exercise 2.2:</b><br>
    Modify the code above for wind speed instead of temperature (hint: you can list the column names in a dataframe with "dataframe.columns").
</div>



## 2.3 Saving/loading objects

Sometimes, we want to save more than just text -- for example, the animals we created above. The details are complicated, but we can save arbitrary objects easily using Pylab:

```python
import sciris as sc
import pylab as pl

class Animal(sc.prettyobj):

    def __init__(self, name, mass, velocity):
        self.name = name
        self.mass = mass
        self.velocity = velocity

    def energy(self):
        return 0.5*self.mass*self.velocity**2

# Create animals
worm   = Animal(name='worm',   mass=0.002, velocity=0.01) # shorthand: Animal('worm', 0.002, 0.01)
pigeon = Animal(name='pigeon', mass=0.086, velocity=9.6)
cat    = Animal(name='cat',    mass=7.3,   velocity=4.4)
animals = [worm, pigeon, cat]

# Save and load animals
pl.save('animals.npy', animals) # file must end .npy (for numpy)
saved_animals = pl.load('animals.npy')

# Check it worked
print(saved_animals[0])
```


## 2.4 To/from Matlab

The `scipy` library comes with support to load matlab files in its `io` package:

```python
import scipy.io as sio
mat_data = sio.loadmat('data/matlab_old.mat')
```

The data is loaded as a Python dictionary, with the variable names as the keys and the variable values as the, well, values. Note that Matlab does not have the concept of one-dimensional arrays, vectors are either row- or column-vectors and therefore two-dimensional (with one of the dimensions having size one):

```python
print(mat_data['X'])
print(mat_data['X'].shape)
```

If you want to avoid this, you can automatically "squeeze" the dimensions when loading:

```python
mat_data = sio.loadmat('data/matlab_old.mat', squeeze_me=True)
print(mat_data['X'].shape)
```

You can also access cell arrays or structures:

```python
print('cell array:', mat_data['c'])
print('name elements of structure s:', mat_data['s']['name'])
```

To save data in the Matlab format, you can use `scipy`'s `savemat` function:

```python
x = np.arange(10)  # some data
y = np.random.rand(10) # some more random data
sio.savemat('file.mat', {'x': x}) # variables have to be given as a dictionary of variable names and values
```

# 3. Advanced plotting

There are many ways of plotting in Python. Matplotlib is the de facto standard, and what we'll focus on here, but for some things, other packages might be better.

## 3.1 Comparison of plotting libraries

The huge variety of plotting libraries is covered by Jake Vanderplas in his 2017 [talk](https://github.com/rougier/python-visualization-landscape). Briefly:

**Matplotlib:** By far the most widely used, should be your go-to unless you have specific requirements otherwise.

```python
import pylab as pl
pl.plot([1,4,3,2,5,8])
pl.title('Simple example')
pl.xlabel('x')
pl.ylabel('y')
```

**Bokeh:** Created by the team behind Anaconda, focuses on visualizing big data interactively. Renders in a browser rather than in Python. A bit more complicated than Matplotlib, but fairly similar.

```python
import bokeh.plotting as bop
p = bop.figure(title="Simple example", x_axis_label='x', y_axis_label='y')
p.line(x=range(6), y=[1,4,3,2,5,8])
bop.show(p)
```

**Altair:** Based on the [Grammar of Graphics](https://www.springer.com/gp/book/9780387245447), steep initial learning curve but then allows complicated statistical data to be plotted intuitively. Input more or less has to be a `pandas` dataframe. Only works from a Jupyter notebook. Not very adaptable to custom plot types.

```python
import altair as alt
import pandas as pd
data = pd.DataFrame({'x': range(6), 'y': [1,4,3,2,5,8]})
alt.Chart(data).mark_line().encode(x='x', y='y').interactive()
```

**Plotnine:** Also based on the Grammar of Graphics, an attempt at porting `ggplot2` from R. Poor documentation, "unpythonic" syntax; input also must be a dataframe.

```python
import plotnine as pn
import pandas as pd
data = pd.DataFrame({'x': range(6), 'y': [1,4,3,2,5,8]})
pn.ggplot(data) + pn.geoms.geom_line()
```

**Plotly:** For-profit company, but managed to get a relatively large user base since it was one of the first packages that allowed plotting in the browser. By default, renders plots on their server (which you need an account for, and in some cases a paid subscription).

```python
import plotly as ply
data = ply.graph_objs.Scatter(x=list(range(6)), y=[1,4,3,2,5,8])
ply.offline.plot({'data':[data]}, auto_open=True)
```

**TLDR:** Use `matplotlib`, unless you want to work interactively with large datasets (`bokeh`), or explore complicated statistical relationships within your highly structured data (`altair`).


## 3.2 Making publication-quality plots

The problem with Matplotlib is that by default the plots it makes are kind of ugly. The simplest way to make them pretty is just to use one of the built-in styles, e.g. adapting our example above:

```python
import pylab as pl
pl.style.use('seaborn')
pl.plot([1,4,3,2,5,8])
pl.title('Simple example')
pl.xlabel('x')
pl.ylabel('y')
```

What makes high-quality plots stand out is typically how much customization is done _for that plot_. Default styles will only get you so far. The following code illustrates some common changes that can make plots look much better: fonts, transparency, spines (whether the plot is in a box or not), legends, and labels. Every aspect of Matplotlib can be customized, so if you can Google it, you can do it!

```python

import pylab as pl

# Simulation parameters
ncells = 1000
tmax = 500
dt = 1
period = 30

# Calculate spikes
pl.seed(1)
spikelist = []
tvec = pl.arange(0,tmax,dt)
firingrate = 5+4*pl.sin(tvec/period)
npts = len(tvec)
for t in range(npts):
    for c in range(ncells):
        if pl.rand() < firingrate[t]*dt/1000:
            spikelist.append([t,c])
spikedata = pl.array(spikelist)

# Average firing rate
averagerate = pl.zeros(npts)
for spike in spikedata:
    averagerate[spike[0]] += 1.0/ncells*1000

# Trend line
trendline = averagerate[:]
for repeat in range(5):
    trendline = pl.convolve(trendline, [0.25, 0.5, 0.25], 'same')

# Set up figure and options
pl.rc('font', family='serif', size=14)
pl.figure(figsize=(16,12))

# Plot the raster
pl.axes([0.1,0.35,0.8,0.6])
pl.scatter(spikedata[:,0], spikedata[:,1], s=100, alpha=0.3)
pl.xlim((0,tmax))
pl.gca().spines['top'].set_visible(False)
pl.gca().spines['right'].set_visible(False)
pl.gca().spines['bottom'].set_visible(False)
pl.xticks([])
pl.ylabel('Cell ID')

# Plot the average firing rate
pl.axes([0.1,0.05,0.8,0.25])
pl.plot(tvec, averagerate, color=(0.7,0.7,0.7), label='Instantaneous')
pl.plot(tvec, trendline, color=(0,0.4,1.0), lw=3, label='Averaged')
pl.plot(tvec, firingrate, '--', color=(0,0,0), lw=2, label='Generator')
pl.xlim((0,tmax))
pl.ylim((0,pl.ylim()[1]))
pl.gca().spines['top'].set_visible(False)
pl.gca().spines['right'].set_visible(False)
pl.xlabel('Time (ms)')
pl.ylabel('Average firing rate (Hz)')
pl.legend(loc=(0.85,0.93))

# Add panel labels
pl.axes([0,0,1,1])
pl.gca().axis('off')
pl.text(x=0.02, y=0.95, s='A', fontsize=35, fontfamily='sans-serif')
pl.text(x=0.02, y=0.3, s='B', fontsize=35, fontfamily='sans-serif')

# Save the figure
pl.savefig('ocnc-example.png', dpi=150)
```

# 4. Exercises


## Exercise 4.1: Izhikevich model

Convert the following Matlab code into Python. Run both and confirm that they produce the same result.

```
% Created by Eugene M. Izhikevich, February 25, 2003
% Excitatory neurons    Inhibitory neurons

% Hint: You will need to import the libraries here (suggestion: import pylab as pl)

Ne=800;                 Ni=200;
re=rand(Ne,1);          ri=rand(Ni,1);
a=[0.02*ones(Ne,1);     0.02+0.08*ri]; % Hint: Use pl.hstack([vector1, vector2])
b=[0.2*ones(Ne,1);      0.25-0.05*ri];
c=[-65+15*re.^2;        -65*ones(Ni,1)];
d=[8-6*re.^2;           2*ones(Ni,1)];
S=[0.5*rand(Ne+Ni,Ne),  -rand(Ne+Ni,Ni)];

v=-65*ones(Ne+Ni,1);    % Initial values of v
u=b.*v;                 % Initial values of u
firings=[];             % spike timings % Hint: use pl.empty((2,0))

for t=1:1000            % simulation of 1000 ms
  I=[5*randn(Ne,1);2*randn(Ni,1)]; % thalamic input
  fired=find(v>=30);    % indices of spikes
  % Hint: you might need an if statement here
  firings=[firings; t+0*fired,fired];
  v(fired)=c(fired);
  u(fired)=u(fired)+d(fired);
  I=I+sum(S(:,fired),2); % Hint: you can specify the axis for sum with .sim(axis=1)
  v=v+0.5*(0.04*v.^2+5*v+140-u+I); % step 0.5 ms
  v=v+0.5*(0.04*v.^2+5*v+140-u+I); % for numerical
  u=u+a.*(b.*v-u);                 % stability
end;
plot(firings(:,1),firings(:,2),'.'); % Hint: use pl.scatter()
% Hint: if the plot doesn't appear, use pl.show() if necessary
```

## Exercise 4.2: Ant simulation

Replace the missing lines with code to run a simulation of ants walking around randomly.

```python
import pylab as pl

class Antsim:
    ''' Simulation of ants randomly walking '''    
    
    def makeants(self, numants=50):
        ''' Initialize the ants '''
        self.numants = numants # Initialize the number of ants
        # Set up the x coordinate: an array of zeros of length numants
        # Set up the y coordinate the same way
    
    def plotants(self, timesteps=150, stepsize=0.03):
        ''' Plot the ants '''
        pl.figure()
        for t in range(timesteps):
            pl.clf()
            # Increment the x position of each ant by an amount stepsize*randn()
            # Increment the y position of each ant by an amount stepsize*randn()
            # Scatter x vs y
            # Set the x limits to (-1,1)
            # Set the y limits to (-1,1)
            # Set the plot title to show the current step and the total number of timesteps
            pl.pause(1e-3)
            

# Run the simulation
sim = Antsim()
sim.makeants(numants=100)
sim.plotants()
```